# Balance Sheet

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
from sklearn.cluster import KMeans
import matplotlib.dates as mdates
from statsmodels.stats.multicomp import pairwise_tukeyhsd

## Financial Accounting

In [2]:
financial_df  = pd.read_csv('../data/double_entry_financial_accounting.csv')
financial_df ['Date'] = pd.to_datetime(financial_df ['Date'])
financial_df ['Date'] = financial_df ['Date'] - pd.DateOffset(years=1)
financial_df .rename(columns={'Transaction_Type': 'Transaction Type', 'Payment_Method': 'Payment Method'}, inplace=True)

In [3]:
financial_df['Balance'] = financial_df['Debit'] - financial_df['Credit']
account_balances = financial_df.groupby('Account')['Balance'].sum()

financial_assets = ['Accounts Receivable', 'Cash', 'Inventory', 'Asset Account']
financial_liabilities = ['Accounts Payable', 'Liability Account']
financial_equity = ['Sales Revenue', 'Expense Account', 'Revenue Account']

financial_assets_df = account_balances[financial_assets].reset_index()
financial_liabilities_df = account_balances[financial_liabilities].reset_index()
financial_equity_df = account_balances[financial_equity].reset_index()

#Reverse the sign of the liabilities and equity because they are on the RHS of the accounting equation
financial_liabilities_df['Balance'] = financial_liabilities_df['Balance'] * -1
financial_equity_df['Balance'] = financial_equity_df['Balance'] * -1

financial_assets_df.columns = ['Account', 'Balance']
financial_liabilities_df.columns = ['Account', 'Balance']
financial_equity_df.columns = ['Account', 'Balance']

print("Assets:")
print(financial_assets_df)
print("\nLiabilities:")
print(financial_liabilities_df)
print("\nEquity:")
print(financial_equity_df)

Assets:
               Account      Balance
0  Accounts Receivable  15272312.94
1                 Cash -13107544.42
2            Inventory   3935363.86
3        Asset Account     24507.99

Liabilities:
             Account     Balance
0   Accounts Payable  1953886.10
1  Liability Account -3717127.19

Equity:
           Account      Balance
0    Sales Revenue  11394054.61
1  Expense Account  -7380479.70
2  Revenue Account   3874306.55


## Managerial Accounting

In [4]:
managerial_df  = pd.read_csv('../data/double_entry_managerial_accounting.csv')
managerial_df['Date'] = pd.to_datetime(managerial_df ['Date'])

In [5]:
managerial_df['Balance'] = managerial_df['Debit'] - managerial_df['Credit']
account_balances = managerial_df.groupby('Account')['Balance'].sum()

managerial_assets = ['Cash', 'Asset Account']
managerial_liabilities = ['Accounts Payable']
managerial_equity = ['Revenue Account', 'Expense Account']

managerial_assets_df = account_balances.reindex(managerial_assets).fillna(0).reset_index()
managerial_liabilities_df = account_balances.reindex(managerial_liabilities).fillna(0).reset_index()
managerial_equity_df = account_balances.reindex(managerial_equity).fillna(0).reset_index()

#Reverse the sign of the liabilities and equity because they are on the RHS of the accounting equation
managerial_liabilities_df['Balance'] = managerial_liabilities_df['Balance'] * -1
managerial_equity_df['Balance'] = managerial_equity_df['Balance'] * -1

managerial_assets_df.columns = ['Account', 'Balance']
managerial_liabilities_df.columns = ['Account', 'Balance']
managerial_equity_df.columns = ['Account', 'Balance']

pd.options.display.float_format = '{:.2f}'.format

print("Assets:")
print(managerial_assets_df)
print("\nLiabilities:")
print(managerial_liabilities_df)
print("\nEquity:")
print(managerial_equity_df)

Assets:
         Account      Balance
0           Cash -54839826.81
1  Asset Account 112400123.56

Liabilities:
            Account  Balance
0  Accounts Payable    -0.00

Equity:
           Account      Balance
0  Revenue Account  77262623.47
1  Expense Account -19702326.72


## Online Retail

In [6]:
online_retail_df  = pd.read_csv('../data/double_entry_online_retail.csv')

In [7]:
online_retail_df['Date'] = pd.to_datetime(online_retail_df['Date'])

online_retail_df['Balance'] = online_retail_df['Debit'] - online_retail_df['Credit']
account_balances = online_retail_df.groupby('Account')['Balance'].sum()

online_retail_assets = ['Accounts Receivable']
online_retail_equity = ['Sales Revenue']

online_retail_assets_df = account_balances.reindex(online_retail_assets).reset_index()
online_retail_equity_df = account_balances.reindex(online_retail_equity).reset_index()

#Reverse the sign of the liabilities and equity because they are on the RHS of the accounting equation
online_retail_equity_df['Balance'] = online_retail_equity_df['Balance'] * -1

online_retail_assets_df.columns = ['Account', 'Balance']
online_retail_equity_df.columns = ['Account', 'Balance']

liabilities_df = pd.DataFrame({'Account': ['No Liabilities'], 'Balance': [0.0]})

print("Assets:")
print(online_retail_assets_df)
print("\nEquity:")
print(online_retail_equity_df)

Assets:
               Account    Balance
0  Accounts Receivable 1559176.95

Equity:
         Account    Balance
0  Sales Revenue 1559176.95


## Final Balance Sheet

In [8]:
combined_df = pd.concat([
    financial_assets_df, financial_liabilities_df, financial_equity_df,
    managerial_assets_df, managerial_liabilities_df, managerial_equity_df,
    online_retail_assets_df, online_retail_equity_df
])

combined_balances = combined_df.groupby('Account')['Balance'].sum().reset_index()

assets_accounts = set(financial_assets + managerial_assets + online_retail_assets)
liabilities_accounts = set(financial_liabilities + managerial_liabilities)
equity_accounts = set(financial_equity + managerial_equity + online_retail_equity)

assets_df = combined_balances[combined_balances['Account'].isin(assets_accounts)]
liabilities_df = combined_balances[combined_balances['Account'].isin(liabilities_accounts)]
equity_df = combined_balances[combined_balances['Account'].isin(equity_accounts)]

print("Assets:")
print(assets_df)
print("\nLiabilities:")
print(liabilities_df)
print("\nEquity:")
print(equity_df)

Assets:
               Account      Balance
1  Accounts Receivable  16831489.89
2        Asset Account 112424631.55
3                 Cash -67947371.23
5            Inventory   3935363.86

Liabilities:
             Account     Balance
0   Accounts Payable  1953886.10
6  Liability Account -3717127.19

Equity:
           Account      Balance
4  Expense Account -27082806.42
7  Revenue Account  81136930.02
8    Sales Revenue  12953231.56
